In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing

from keras import models
from keras import layers
from keras import optimizers
from keras.layers import Activation
from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.


In [2]:
# Create the model
# Add convolution and pooling layers
model = models.Sequential()
model.add(layers.Conv2D(64, (11,11), activation='relu', input_shape=(128, 128, 3)))
model.add(layers.MaxPooling2D(pool_size=(2,2), strides=2))
model.add(layers.Dropout(0.1))

model.add(layers.Conv2D(128, (11,11), activation='relu')) 
model.add(layers.MaxPooling2D(pool_size=(2,2), strides=2))
model.add(layers.Dropout(0.1))

model.add(layers.Conv2D(256, (11,11), activation='relu')) 
model.add(layers.MaxPooling2D(pool_size=(2,2), strides=2))
model.add(layers.Dropout(0.2))

# Add fully connected layers
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(6, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 118, 118, 64)      23296     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 59, 59, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 59, 59, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 49, 49, 128)       991360    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 24, 24, 128)       0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 24, 24, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 256)      

In [3]:
# Cofnigure paths
import os
base_dir = '/kaggle/input/duth-cv-2019-2020-hw-4/vehicles'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen  = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=20,
                                                    class_mode='categorical',
                                                    target_size=(128,128),
                                                    shuffle=True)     
# Flow validation images in batches of 20 using val_datagen generator
validation_generator =  val_datagen.flow_from_directory(validation_dir,
                                                        batch_size=20,
                                                        class_mode='categorical',
                                                         target_size=(128,128)) 

Found 2494 images belonging to 6 classes.
Found 311 images belonging to 6 classes.


In [5]:
from keras.callbacks import ModelCheckpoint

# Define callback that saves the best epoch
mcp_save = ModelCheckpoint('best_epoch.h5', save_best_only=True, monitor='val_loss', mode='min', verbose=1)

In [6]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=5e-5),
              metrics=['acc'])

# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      callbacks=[mcp_save],
      verbose=1)

Epoch 1/100
125/124 [==============================] - 20s 157ms/step - loss: 1.5740 - acc: 0.3508 - val_loss: 1.5323 - val_acc: 0.3537

Epoch 00001: val_loss improved from inf to 1.53225, saving model to best_epoch.h5
Epoch 2/100
125/124 [==============================] - 14s 111ms/step - loss: 1.3761 - acc: 0.4463 - val_loss: 1.1194 - val_acc: 0.4791

Epoch 00002: val_loss improved from 1.53225 to 1.11940, saving model to best_epoch.h5
Epoch 3/100
125/124 [==============================] - 14s 114ms/step - loss: 1.1813 - acc: 0.5545 - val_loss: 1.0256 - val_acc: 0.5691

Epoch 00003: val_loss improved from 1.11940 to 1.02555, saving model to best_epoch.h5
Epoch 4/100
125/124 [==============================] - 14s 112ms/step - loss: 1.0962 - acc: 0.5842 - val_loss: 1.4968 - val_acc: 0.5305

Epoch 00004: val_loss did not improve from 1.02555
Epoch 5/100
125/124 [==============================] - 14s 113ms/step - loss: 1.0210 - acc: 0.6151 - val_loss: 0.6145 - val_acc: 0.6013

Epoch 0000

In [7]:
import tensorflow as tf
from keras.preprocessing import image
import csv

model = tf.keras.models.load_model('/kaggle/working/best_epoch.h5')
rowlist = [['Id', 'Category']]

for dirname, _, filenames in os.walk('/kaggle/input/duth-cv-2019-2020-hw-4/vehicles/test'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        img = image.load_img(path, target_size=(128, 128), grayscale=False, interpolation='bilinear')
        
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        
        classes_pred = model.predict(x)
        cls_pred = np.argmax(classes_pred)
        rowlist.append([filename, cls_pred])
        with open('output.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(rowlist)